In [2]:
import folium
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [ ]:
df = pd.read_csv("../../test_full_pattern.csv")

test_one = df[df["unique_trip_vehicle_day"]== "7295.02353141648835741081142023-01-03"]

test_one['location'] = test_one['location'].astype(str)
test_one['geometry'] = test_one['location'].apply(wkt.loads)

test_one = gpd.GeoDataFrame(test_one, geometry='geometry', crs='epsg:4326')

In [29]:
bus = test_one[test_one['typ'] == 'B']
bus = gpd.GeoSeries(bus["geometry"])
bus = bus.to_json()

stop = test_one[test_one['typ'] == 'S']
stop = gpd.GeoSeries(stop["geometry"])
stop = stop.to_json()

other = test_one[test_one['typ'] == 'W']
other = gpd.GeoSeries(other["geometry"])
other = other.to_json()

In [21]:
# prep segments
segments_df = pd.read_parquet("../cta-stop-etl/out/patterns/pid_95_segment.parquet")
segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:26971')

pattern_segments_95 = gpd.GeoSeries(segments_df["geometry"])
pattern_segments_95 = pattern_segments_95.to_json()

In [33]:


m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)


# add one trip bus location
folium.GeoJson(data=bus, marker=folium.Circle(
                   radius=50,
                   fill_color="red",
                   fill_opacity=0.4,
                   color="black",
                   weight=1), 
                   name = 'One Bus Trip').add_to(m)

# add bus stops
folium.GeoJson(data=stop, marker=folium.Circle(
                   radius=30,
                   fill_color="green",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Bus Stops').add_to(m)

folium.GeoJson(data=other, marker=folium.Circle(
                   radius=10,
                   fill_color="blue",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Other Bus Routes Waypoints').add_to(m)



# add segments
folium.GeoJson(data=pattern_segments_95, style_function=lambda x: {"fillColor": "grey", "weight": '1'}, name = 'Route with Buffer').add_to(m)

folium.LayerControl().add_to(m)

m